In [ ]:
import pandas as pd
import time
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models

In [ ]:
type = 'type' #zero_shot; #few_shot...
output_path = f'''output_path/{type}.csv'''
dataset_path = "dataset_path/"
prompt_path = "prompt_path/"

In [ ]:
def generate(prompt:str, max_tokens:int, temperature:float):
  vertexai.init(project="hate-speech-pt-416320", location="us-central1")
  model = GenerativeModel("gemini-1.0-pro-001")
  responses = model.generate_content(
    prompt,
    generation_config={
        "max_output_tokens": max_tokens,
        "temperature": temperature,
        "top_p": 1
    },
    safety_settings={
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_NONE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_NONE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    },
    stream=False,
  )
  
  return responses

In [ ]:
base_prompt = ''
with open(prompt_path, 'r') as file:
    base_prompt = file.read()

print(base_prompt)

In [ ]:
df = pd.read_csv(dataset_path)

df.head()

In [ ]:
answers = ["none"] * len(df)

In [ ]:
i = 0

while i < len(df):
  try:
    prompt = base_prompt +  f'''\nTexto: {df["text"].iloc[i]}\nClasse: '''

    resp = generate(prompt, 20, 0.1)

    resp_texto = resp.text.replace("\n", '')
    answers[i] = resp_texto.lower()
    
    i += 1

    time.sleep(2)

  except Exception as e:
    erro = f"Erro encontrado: {e}. Tentando novamente..."
    print(erro)
    
    answers[i] = erro

    i += 1
    
    time.sleep(5)  # Pausa antes de tentar novamente

In [ ]:
df_answers = pd.DataFrame({
    "text": df['text'].values,
    "answer": answers,
})

df_answers.to_csv(output_path, index=False)